# California Housing Price Prediction con Amazon SageMaker Autopilot 

El kernel `Python 3 (Data Science)` funciona bien con este notebook dentro de SageMaker Studio.

---
## Contenido

1. [Introducción](#Introduction)
1. [Setup](#Setup)
1. [Preparación de los datos de entrenamiento](#Data)
1. [Entrenamiento](#Settingup)
1. [Lanzando el job de SageMaker Autopilot](#Results)
1. [Evaluación utilizando los datos de test](#Evaluate)
1. [Cleanup](#Cleanup)


---

### Introducción<a name="Introduction"></a>

Amazon SageMaker Autopilot es una solución de aprendizaje automático (comúnmente conocida como AutoML) para conjuntos de datos tabulares. Podemos utilizar SageMaker Autopilot de diferentes maneras:

- en piloto automático (sin ninguna entrada humana) 
- con orientación humana
- sin código a través de SageMaker Studio
- con scripts utilizando los AWS SDK.

Este cuaderno utilizará los SDKs de AWS para crear e implementar de forma sencilla un modelo de aprendizaje automático sin hacer feature engineering manualmente. También exploraremos el informe de importancia de features autogenerado.

La predicción de los precios de la vivienda es un problema clásico de regresión lineal en ML. El Dataset describe a las viviendas que se encuentran en un determinado distrito de California (como grupo) e incluye estadísticas resumidas sobre ellas basadas en los datos del censo de 1990.

Las variables del conjunto de datos son fácilmente comprensibles, y las columnas son las siguientes:

* ```longitude```
* ```latitude```
* ```housingMedianAge```
* ```totalRooms```
* ```totalBedrooms```
* ```population```
* ```households```
* ```medianIncome```
* ```medianHouseValue``` (target)

Lo que vamos a intentar predecir es el valor medio de la vivienda en un distrito. Dejaremos que Autopilot realice feature engineering, model selection, tuning de los mismos y nos dé el mejor modelo candidato listo para utilizarlo en las inferencias.

---
## Setup

_Este notebook fue creado y probado en una instancia `ml.t3.medium`._

Empecemos por especificar:

- El bucket de S3 y el prefijo a utilizar para los datos de entrenamiento y del modelo.  Este debería estar dentro de la misma región que la instancia de Notebook, el entrenamiento y el alojamiento. El siguiente código utilizará el bucket de S3 por defecto de SageMaker (o creará uno si no existe).
- El ARN del rol IAM utilizado para dar acceso a los datos. El siguiente código utilizará el rol de ejecución de SageMaker.

In [2]:
import sagemaker
import boto3
from sagemaker import get_execution_role

region = boto3.Session().region_name

session = sagemaker.Session()

bucket = session.default_bucket()
prefix = "sagemaker/DEMO-autopilot-housing"

role = get_execution_role()

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

A continuación, importaremos las bibliotecas de Python que necesitaremos para el resto del ejercicio.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
from urllib.parse import urlparse
import time

---
## Preparación de los datos de entrenamiento <a name="Data"></a>

Utilizaremos el Dataset de vivienda de California, de acceso público. La variable objetivo es el valor medio de la vivienda en los distritos de California.

Este Dataset se obtuvo del repositorio StatLib (http://lib.stat.cmu.edu/datasets/) y se derivó del censo de EE.UU. de 1990, utilizando una fila por grupo de bloques del censo. Un grupo de bloques es la unidad geográfica más pequeña para la que la Oficina del Censo de EE.UU. publica datos de muestra (un grupo de bloques suele tener una población de entre 600 y 3.000 personas).

Descargamos y extraemos el Dataset en el EFS montado en el Notebook.

In [4]:
!aws s3 cp s3://sagemaker-sample-files/datasets/tabular/california_housing/cal_housing.tgz .
!tar -zxf cal_housing.tgz -o

download: s3://sagemaker-sample-files/datasets/tabular/california_housing/cal_housing.tgz to ./cal_housing.tgz


In [6]:
columns = [
    "longitude",
    "latitude",
    "housingMedianAge",
    "totalRooms",
    "totalBedrooms",
    "population",
    "households",
    "medianIncome",
    "medianHouseValue",
]

target = "medianHouseValue"

cal_housing_df = pd.read_csv("CaliforniaHousing/cal_housing.data", names=columns, header=None)

Antes de iniciar un trabajo de AutoML, es una buena práctica inspeccionar los datos para asegurarse de que no hay problemas obvios con ellos.

In [7]:
pd.set_option("display.max_columns", 500)
cal_housing_df

,longitude,latitude,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0
...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0


Observa que las distintas columnas tienen valores en rangos diferentes. Por ejemplo, los valores de la columna objetivo ``medianHouseValue`` son órdenes de magnitud superiores a los de otras columnas. Esta diferencia en la escala a menudo causa problemas cuando se entrena un modelo ML, por lo que es una práctica común hacer feature engineering para normalizar o estandarizar esos valores (dependiendo de la naturaleza de la distribución numérica y la presencia de valores atípicos).

Sin embargo, dado que Autopilot se encarga de hacer feature engineering automáticamente (entre otras cosas), no vamos a realizar ningún análisis o transformación nosotros mismos.

Para ilustrar también cómo Autopilot maneja problemas en los datos, como los valores nulos, introduzcamos algunos valores vacíos aleatorios en nuestro conjunto de datos para la columna de la edad media de la vivienda.


In [8]:
# Tomamos una pequeña muestra aleatoriamente del Dataset
cal_housing_df_sample = cal_housing_df.sample(frac=0.01, random_state=100)

cal_housing_df["housingMedianAge"].loc[cal_housing_df_sample.index] = (
    cal_housing_df["housingMedianAge"].loc[cal_housing_df_sample.index].apply(lambda row: np.nan)
)

Ahora el Dataset debería contener valores nulos para ~1% de las filas en la columna `housingMedianAge`.

In [9]:
isna_sum = cal_housing_df["housingMedianAge"].isna().sum()

print(f"{isna_sum} values removed from housingMedianAge column")

206 values removed from housingMedianAge column


Dividimos los datos para entrenamiento y testing (80/20 split)

In [10]:
train_data = cal_housing_df.sample(frac=0.8, random_state=200)

test_data = cal_housing_df.drop(train_data.index)

test_data_no_target = test_data.drop(columns=[target])

Subimos el Dataset a S3

In [11]:
train_file = "train_data.csv"
train_data.to_csv(train_file, index=False, header=True)
train_data_s3_path = session.upload_data(path=train_file, key_prefix=prefix + "/train")
print("Train data uploaded to: " + train_data_s3_path)

test_file = "test_data_no_target.csv"
test_data_no_target.to_csv(test_file, index=False, header=False)
test_data_s3_path = session.upload_data(path=test_file, key_prefix=prefix + "/test")
print("Test data uploaded to: " + test_data_s3_path)

Train data uploaded to: s3://sagemaker-us-east-1-688013747199/sagemaker/DEMO-autopilot-housing/train/train_data.csv
Test data uploaded to: s3://sagemaker-us-east-1-688013747199/sagemaker/DEMO-autopilot-housing/test/test_data_no_target.csv


---

## Setup del job de SageMaker <a name="Settingup"></a>


After uploading the dataset to Amazon S3, you can invoke Autopilot to find the best ML pipeline to train a model on this dataset. 

The required inputs for invoking an Autopilot job are:
* Amazon S3 location for input dataset and for all output artifacts
* Name of the target column in the dataset for predictions 
* An IAM role

Currently, Autopilot supports only tabular datasets in CSV format. Either all files should have a header row, or the first file of the dataset, when sorted in alphabetical/lexical order by name, is expected to have a header row.


Después de cargar el Dataset en Amazon S3, podemos invocar a Autopilot para encontrar el mejor pipeline para entrenar un modelo con dicho Dataset.

Los inputs necesarios para invocar un job de Autopilot son:

* URI de Amazon S3 para el Dataset de entrada y para todos los artefactos de salida.
* Nombre de la columna target en el Dataset para las predicciones
* Un rol IAM

Actualmente, Autopilot solo admite Datasets tabulares en formato CSV. O bien todos los archivos deben tener una fila header, o bien se espera que el primer archivo del conjunto de datos, cuando se clasifica en orden alfabético/léxico por nombre, tenga una fila header.


In [ ]:
input_data_config = [
    {
        "DataSource": {
            "S3DataSource": {
                "S3DataType": "S3Prefix",
                "S3Uri": f"s3://{bucket}/{prefix}/train",
            }
        },
        "TargetAttributeName": target,
    }
]

job_config = {
    "CompletionCriteria": {
        "MaxCandidates": 2,
        # "MaxRuntimePerTrainingJobInSeconds": 300,
        "MaxAutoMLJobRuntimeInSeconds": 60 * 60
    }
}


output_data_config = {"S3OutputPath": f"s3://{bucket}/{prefix}/output"}

: 

También podemos especificar el tipo de problema que queremos resolver con el Dataset (`Regresión, Multiclasificación, Clasificación Binaria`). En caso de que no estemos seguro, SageMaker Autopilot inferirá el tipo de problema basado en las estadísticas de la columna target.

Como el atributo target `medianHouseValue` es una variable numérica continua, el AutoPilot inferirá el tipo de problema como regresión.

Tenemos la opción de limitar el tiempo de ejecución de un trabajo de SageMaker Autopilot proporcionando el número máximo de pipelines de evaluació o candidatos (un pipeline de evaluación se llama `Candidate` porque genera un modelo candidato) o proporcionando el tiempo total asignado para el trabajo general de Autopilot. Con la configuración predeterminada, este trabajo puede tardar varias horas en ejecutarse. Esto varía entre ejecuciones debido a la naturaleza del proceso exploratorio que Autopilot utiliza para encontrar los parámetros óptimos de entrenamiento.

Para esta demostración, limitamos el número de candidatos a 2 para que el trabajo termine en menos de 1 hora.

Por último, también podemos desplegar el modelo que resulte "ganador" en un endpoint de SageMaker automáticamente al terminar. En este caso, no desplegaremos el endpoint. En su lugar, ejecutaremos un trabajo de predicción por lotes para evaluar nuestro modelo.

Para obtener orientación sobre cómo configurar los parámetros del trabajo, podemos consultar la documentación del SDK.

### Lanzando el job de SageMaker Autopilot<a name="Launching"></a>

Ahora puede lanzar el job de AutoPilot llamando a la API `create_auto_ml_job`.

In [23]:
from time import gmtime, strftime, sleep

timestamp_suffix = strftime("%Y%m%d-%H-%M", gmtime())

auto_ml_job_name = "automl-housing-" + timestamp_suffix
print("AutoMLJobName: " + auto_ml_job_name)

sm.create_auto_ml_job(
    AutoMLJobName=auto_ml_job_name,
    InputDataConfig=input_data_config,
    OutputDataConfig=output_data_config,
    AutoMLJobConfig=job_config,
    # Uncomment to automatically deploy an endpoint
    # ModelDeployConfig={
    #'AutoGenerateEndpointName': True,
    #'EndpointName': 'autopilot-DEMO-housing-' + timestamp_suffix
    # },
    RoleArn=role,
)

AutoMLJobName: automl-housing-20221116-19-20


{'AutoMLJobArn': 'arn:aws:sagemaker:us-east-1:688013747199:automl-job/automl-housing-20221116-19-20',
 'ResponseMetadata': {'RequestId': '2e4f7058-516e-4f0e-8186-48f244834132',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2e4f7058-516e-4f0e-8186-48f244834132',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '100',
   'date': 'Wed, 16 Nov 2022 19:20:29 GMT'},
  'RetryAttempts': 0}}

---
El job de AutoPilot realizará ahora los siguientes pasos:

* Análisis de datos
* Ingeniería de features
* Selección del modelo
* Ajuste del modelo (optimización de hiperparámetros)
* Modelado de la importancia de las features del modelo (SageMaker Clarify)


### Siiguiendo el progreso del job de SageMaker Autopilot

In [24]:
print("JobStatus - Secondary Status")
print("------------------------------")


describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
print(describe_response["AutoMLJobStatus"] + " - " + describe_response["AutoMLJobSecondaryStatus"])
job_run_status = describe_response["AutoMLJobStatus"]

while job_run_status not in ("Failed", "Completed", "Stopped"):
    describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    job_run_status = describe_response["AutoMLJobStatus"]

    print(
        describe_response["AutoMLJobStatus"] + " - " + describe_response["AutoMLJobSecondaryStatus"]
    )
    sleep(60)

JobStatus - Secondary Status
------------------------------
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - ModelTuning
InProgress - ModelTuning
InProgress - ModelTuning
InProgress - MergingAutoMLTaskReports
InProgress - MergingAutoMLTaskReports
InProgress - MergingAutoMLTaskReports
InProgress - MergingAutoMLTaskReports
InProgress - MergingAutoMLTaskReports
InProgress - MergingAutoMLTaskReports
InProgress - MergingAutoMLTaskReports
InProgress - MergingAutoMLTaskReports
InP

---
## Resultados

Ahora podemos utilizar la API `describe_auto_ml_job` para buscar el mejor candidato seleccionado por el trabajo de SageMaker Autopilot.

In [25]:
best_candidate = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)["BestCandidate"]
best_candidate_name = best_candidate["CandidateName"]

print("\n")
print("CandidateName: " + best_candidate_name)
print(
    "FinalAutoMLJobObjectiveMetricName: "
    + best_candidate["FinalAutoMLJobObjectiveMetric"]["MetricName"]
)
print(
    "FinalAutoMLJobObjectiveMetricValue: "
    + str(best_candidate["FinalAutoMLJobObjectiveMetric"]["Value"])
)
print("\nBest candidate details:: " + str(best_candidate))



CandidateName: automl-housing-20221116-19-20uij-002-d2d17f5c
FinalAutoMLJobObjectiveMetricName: validation:mse
FinalAutoMLJobObjectiveMetricValue: 2977769472.0

Best candidate details:: {'CandidateName': 'automl-housing-20221116-19-20uij-002-d2d17f5c', 'FinalAutoMLJobObjectiveMetric': {'MetricName': 'validation:mse', 'Value': 2977769472.0}, 'ObjectiveStatus': 'Succeeded', 'CandidateSteps': [{'CandidateStepType': 'AWS::SageMaker::ProcessingJob', 'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:688013747199:processing-job/automl-housing-20221116-19-20-db-1-98da5f6194174a48a3c22f59cc59', 'CandidateStepName': 'automl-housing-20221116-19-20-db-1-98da5f6194174a48a3c22f59cc59'}, {'CandidateStepType': 'AWS::SageMaker::TrainingJob', 'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:688013747199:training-job/automl-housing-20221116-19-20-dpp1-1-bf59a858abe4446a90b1c77c03', 'CandidateStepName': 'automl-housing-20221116-19-20-dpp1-1-bf59a858abe4446a90b1c77c03'}, {'CandidateStepType': 'AWS::SageMa

Si sentimos curiosidad por explorar el rendimiento de los otros algoritmos explorados por AutoPilot, podemos enumerarlos llamando a la API `list_candidates_for_auto_ml_job`.

In [26]:
sm_dict = sm.list_candidates_for_auto_ml_job(AutoMLJobName=auto_ml_job_name)
for item in sm_dict["Candidates"]:
    print(item["CandidateName"], item["FinalAutoMLJobObjectiveMetric"])
    print(item["InferenceContainers"][1]["Image"], "\n")

automl-housing-20221116-19-20uij-002-d2d17f5c {'MetricName': 'validation:mse', 'Value': 2977769472.0}
683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.3-1-cpu-py3 

automl-housing-20221116-19-20uij-001-4b62271a {'MetricName': 'validation:mse', 'Value': 3039514368.0}
683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.3-1-cpu-py3 



---
AutoPilot genera automáticamente dos Jupyter Notebooks:  

- `SageMakerAutopilotDataExplorationNotebook.ipynb`
- `SageMakerAutopilotCandidateDefinitionNotebook.ipynb`

Estos notebooks se guardan en S3. Vamos a descargarlos en nuestra instancia de SageMaker para explorarlos.

In [27]:
candidate_nbk = describe_response["AutoMLJobArtifacts"]["CandidateDefinitionNotebookLocation"]
data_explore_nbk = describe_response["AutoMLJobArtifacts"]["DataExplorationNotebookLocation"]


def split_s3_path(s3_path):
    path_parts = s3_path.replace("s3://", "").split("/")
    bucket = path_parts.pop(0)
    key = "/".join(path_parts)
    return bucket, key


s3_bucket, candidate_nbk_key = split_s3_path(candidate_nbk)
_, data_explore_nbk_key = split_s3_path(data_explore_nbk)

print(s3_bucket, candidate_nbk_key, data_explore_nbk_key)

session.download_data(path="./", bucket=s3_bucket, key_prefix=candidate_nbk_key)

session.download_data(path="./", bucket=s3_bucket, key_prefix=data_explore_nbk_key)

sagemaker-us-east-1-688013747199 sagemaker/DEMO-autopilot-housing/output/automl-housing-20221116-19-20/sagemaker-automl-candidates/automl-housing-20221116-19-20-pr-1-494dabd5d26444208726a619d64f/notebooks/SageMakerAutopilotCandidateDefinitionNotebook.ipynb sagemaker/DEMO-autopilot-housing/output/automl-housing-20221116-19-20/sagemaker-automl-candidates/automl-housing-20221116-19-20-pr-1-494dabd5d26444208726a619d64f/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb


#### Notebooks candidatos durante la exploración de datos

**Vamos a inspeccionar el cuaderno de exploración de datos. Comprueba la sección `Column Analysis and Descriptive Statistics` para ver el análisis realizado por Autopilot.**

**Ahora inspeccionemos el notebook Candidate Generation. Comprueba la sección  `Generated Candidates` para ver los diferentes algoritmos y estrategias de transformación de datos utilizados por Autopilot.**

---
Autopilot también genera automáticamente un reporte sobre la importancia de las features.


In [28]:
explainability_prefix = best_candidate["CandidateProperties"]["CandidateArtifactLocations"][
    "Explainability"
]

s3_bucket, explainability_dir = split_s3_path(explainability_prefix)

session.download_data(path="./", bucket=s3_bucket, key_prefix=explainability_dir)

El código anterior descargará un directorio en nuestro entorno local. En ese directorio (el prefijo es el nombre del job de AutoPilot, el sufijo se genera automáticamente), debería ver los artefactos de SageMaker Clarify. SageMaker Clarify proporciona una mayor visibilidad de los datos y modelos de entrenamiento para identificar y limitar el sesgo y explicar las predicciones. El siguiente código abrirá el informe de importancia de la característica:

In [29]:
from IPython.display import IFrame

# Obtener el nombre de directorio autogenerado para los artefactos de SageMaker Clarify
dir_name = (
    session.list_s3_files(bucket=s3_bucket, key_prefix=explainability_dir)[0]
    .replace(explainability_dir, "")
    .split("/")[1]
)

# Mostrar informe HTML
IFrame(src=f"{dir_name}/report.html", width=700, height=600)

Your results may vary. But you're likely to see latitude and longitude (i.e., location) on top, along with population size and median income, which are stronger predictors of housing prices than the other features in the dataset.

In SageMaker Studio, you can also navigate to SageMaker resources tab, click on Experiments and trials, and find your Autopilot experiment. You can double-click on the experiment name to list all trials, and from there you can double-click on a specific trial to see its details, including charts and metrics.

---
## Evaluate Model Using Test Dataset<a name="Evaluate"></a>

To evaluate the model on previously unseen data, we will test it against the test dataset we prepared earlier. For that, we don't necessarily need to deploy the model to an endpoint, we can simply run a batch transform job to get predictions for our unlabeled test dataset.

### Set up Transform Job

In [30]:
from sagemaker import AutoML

automl = AutoML.attach(auto_ml_job_name=auto_ml_job_name)

s3_transform_output_path = "s3://{}/{}/inference-results/".format(s3_bucket, prefix)

model_name = "{0}-model".format(best_candidate_name)

model = automl.create_model(
    name=model_name,
    candidate=best_candidate,
)

output_path = s3_transform_output_path + best_candidate_name + "/"

transformer = model.transformer(
    instance_count=1,
    instance_type="ml.m5.xlarge",
    assemble_with="Line",
    strategy="SingleRecord",
    output_path=output_path,
    env={"SAGEMAKER_MODEL_SERVER_TIMEOUT": "100", "SAGEMAKER_MODEL_SERVER_WORKERS": "1"},
)

### Launch Transform Job

In [31]:
transformer.transform(
    data=test_data_s3_path,
    split_type="Line",
    content_type="text/csv",
    wait=False,
    model_client_config={"InvocationsTimeoutInSeconds": 80, "InvocationsMaxRetries": 1},
)

print("Starting transform job {}".format(transformer._current_job_name))

Starting transform job automl-housing-20221116-19-20uij-002-d2-2022-11-16-20-00-33-113


#### Track Transform Job Status

In [32]:
## Wait for jobs to finish
pending_complete = True
job_name = transformer._current_job_name

while pending_complete:
    pending_complete = False

    description = sm.describe_transform_job(TransformJobName=job_name)
    if description["TransformJobStatus"] not in ["Failed", "Completed"]:
        pending_complete = True

    print("{} transform job is running.".format(job_name))
    time.sleep(60)

print("\nCompleted.")

automl-housing-20221116-19-20uij-002-d2-2022-11-16-20-00-33-113 transform job is running.
automl-housing-20221116-19-20uij-002-d2-2022-11-16-20-00-33-113 transform job is running.
automl-housing-20221116-19-20uij-002-d2-2022-11-16-20-00-33-113 transform job is running.
automl-housing-20221116-19-20uij-002-d2-2022-11-16-20-00-33-113 transform job is running.
automl-housing-20221116-19-20uij-002-d2-2022-11-16-20-00-33-113 transform job is running.
automl-housing-20221116-19-20uij-002-d2-2022-11-16-20-00-33-113 transform job is running.
automl-housing-20221116-19-20uij-002-d2-2022-11-16-20-00-33-113 transform job is running.

Completed.


### Evaluate the Inference Results

The transform job will have now generated a CSV file with inference results for the test dataset. We will use those results and compare them with the real test labels to see how the model performs compared to real data.

In [33]:
def get_csv_from_s3(s3uri, file_name):
    parsed_url = urlparse(s3uri)
    bucket_name = parsed_url.netloc
    prefix = parsed_url.path[1:].strip("/")
    s3 = boto3.resource("s3")
    obj = s3.Object(bucket_name, "{}/{}".format(prefix, file_name))
    return obj.get()["Body"].read().decode("utf-8")


job_status = sm.describe_transform_job(TransformJobName=job_name)["TransformJobStatus"]

if job_status == "Completed":
    pred_csv = get_csv_from_s3(transformer.output_path, "{}.out".format(test_file))
    predictions = pd.read_csv(io.StringIO(pred_csv), header=None)

In [34]:
from sklearn.metrics import mean_squared_error
from math import sqrt

labels_df = test_data[target]
mse = mean_squared_error(labels_df, predictions)
rmse = sqrt(mse)

print("MSE: {0}\nRMSE: {1}".format(mse, rmse))

MSE: 2035430611.447957
RMSE: 45115.746823564354


---
## Cleanup

The Autopilot job creates many underlying artifacts such as dataset splits, preprocessing scripts, or preprocessed data, etc. This code, when un-commented, deletes them. 

In [ ]:
s3 = boto3.resource("s3")
s3_bucket = s3.Bucket(bucket)

print(s3_bucket)
job_outputs_prefix = "{}/output/{}".format(prefix, auto_ml_job_name)
print(job_outputs_prefix)

# Delete S3 objects
s3_bucket.objects.filter(Prefix=job_outputs_prefix).delete()

We then delete all the experiment and model resources created by the Autopilot experiment.

In [ ]:
def cleanup_experiment_resources(experiment_name):
    trials = sm.list_trials(ExperimentName=experiment_name)["TrialSummaries"]
    print("TrialNames:")
    for trial in trials:
        trial_name = trial["TrialName"]
        print(f"\n{trial_name}")

        components_in_trial = sm.list_trial_components(TrialName=trial_name)
        print("\tTrialComponentNames:")
        for component in components_in_trial["TrialComponentSummaries"]:
            component_name = component["TrialComponentName"]
            print(f"\t{component_name}")
            sm.disassociate_trial_component(TrialComponentName=component_name, TrialName=trial_name)
            try:
                # comment out to keep trial components
                sm.delete_trial_component(TrialComponentName=component_name)
            except:
                # component is associated with another trial
                continue
            # to prevent throttling
            time.sleep(5)
        sm.delete_trial(TrialName=trial_name)
    sm.delete_experiment(ExperimentName=experiment_name)
    print(f"\nExperiment {experiment_name} deleted")


def cleanup_autopilot_models(autopilot_job_name):
    print("{0}:\n".format(autopilot_job_name))
    response = sm.list_models(NameContains=autopilot_job_name)

    for model in response["Models"]:
        model_name = model["ModelName"]
        print(f"\t{model_name}")
        sm.delete_model(ModelName=model_name)
        # to prevent throttling
        time.sleep(3)

In [ ]:
cleanup_experiment_resources("{0}-aws-auto-ml-job".format(auto_ml_job_name))

In [ ]:
cleanup_autopilot_models(auto_ml_job_name)

Finally, the following code, when uncommented, will delete the local files used in this demo.

In [ ]:
import shutil
import glob
import os


def delete_local_files():
    base_path = ""
    dir_list = glob.iglob(os.path.join(base_path, "{0}*".format(auto_ml_job_name)))

    for path in dir_list:
        if os.path.isdir(path):
            shutil.rmtree(path)

    if os.path.exists("CaliforniaHousing"):
        shutil.rmtree("CaliforniaHousing")

    if os.path.exists("cal_housing.tgz"):
        os.remove("cal_housing.tgz")

    if os.path.exists("SageMakerAutopilotCandidateDefinitionNotebook.ipynb"):
        os.remove("SageMakerAutopilotCandidateDefinitionNotebook.ipynb")

    if os.path.exists("SageMakerAutopilotDataExplorationNotebook.ipynb"):
        os.remove("SageMakerAutopilotDataExplorationNotebook.ipynb")

    if os.path.exists("test_data_no_target.csv"):
        os.remove("test_data_no_target.csv")

    if os.path.exists("test_data.csv"):
        os.remove("test_data.csv")

    if os.path.exists("train_data.csv"):
        os.remove("train_data.csv")


## UNCOMMENT TO CLEAN UP LOCAL FILES
# delete_local_files()

**Note: If you enabled automatic endpoint creation, you will need to delete the endpoint manually.**